In [117]:
import pandas as pd
import numpy as np

In [118]:
data=pd.read_csv('data/spam.csv')
data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [119]:
data.columns

Index(['Category', 'Message'], dtype='object')

In [120]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [121]:
data.isna().sum()

Category    0
Message     0
dtype: int64

In [122]:
data = data.drop_duplicates()

In [123]:
data['Spam']=data['Category'].apply(lambda x:1 if x=='spam' else 0)
data.head(5)

C:\Users\yardz\AppData\Local\Temp\ipykernel_4840\1085243151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Spam']=data['Category'].apply(lambda x:1 if x=='spam' else 0)


,Category,Message,Spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Message'])
y = data['Spam']

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Випадковий пошук параметрів
random_grid = {
    'n_estimators': np.arange(10, 300, 10),  # Великий діапазон
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}
rf_model = RandomForestClassifier(random_state=42)

# RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model,
                                   param_distributions=random_grid,
                                   n_iter=50,  # Кількість випадкових комбінацій
                                   cv=3,
                                   verbose=2,
                                   random_state=42,
                                   n_jobs=-1)

# Навчання
random_search.fit(X_train, y_train)

model = random_search.best_estimator_
# Найкращі параметри
print("Best Hyperparameters (Randomized Search):", random_search.best_params_)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Hyperparameters (Randomized Search): {'n_estimators': np.int64(240), 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}


In [126]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
scores = cross_val_score(model, X, y, cv=5)
print(classification_report(y_test, y_pred))
print('Avg Accuracy:', scores.mean())

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       896
           1       1.00      0.84      0.91       136

    accuracy                           0.98      1032
   macro avg       0.99      0.92      0.95      1032
weighted avg       0.98      0.98      0.98      1032

Avg Accuracy: 0.97130016015158


In [127]:
import joblib
joblib.dump(model, 'model2.pkl')

['model2.pkl']